# 第6章　ファクターモデルの導入

In [ ]:
using CSV, DataFrames, DataFramesMeta, ShiftedArrays, Statistics, Gadfly, CategoricalArrays

In [ ]:
#ch06_01
monthly_data_df = DataFrame(CSV.File("../../data/ch05_output1.csv", missingstring=["NA"]))
annual_data_df = DataFrame(CSV.File("../../data/ch05_output2.csv", missingstring=["NA"]))

In [ ]:
#ch06_02
annual_data_df = @chain annual_data_df begin
    @orderby(:firm_ID, :year)
    groupby(:firm_ID)
    @transform(:lagged_ME = lag(:ME))
end

In [ ]:
#ch06_03
@chain annual_data_df begin
    groupby(:year)
    @transform(:w_M = :lagged_ME ./ sum(skipmissing(:lagged_ME)))
    first(10)
    println
end

In [ ]:
#ch06_04
mask1 = annual_data_df.year .>= 2016
mask2 = ismissing(annual_data_df.w_M)
mask = mask1 .&& mask2
#annual_data_df[mask, :w_M] .= 0
annual_data_df[mask, :]

In [ ]:
#ch06_05
@chain annual_data_df begin
    groupby(:year)
    @combine(:weight_sum = sum(:w_M))
end

In [ ]:
#ch06_06
monthly_data_df = @chain annual_data_df begin
    @select(:year, :firm_ID, :w_M)
    outerjoin(monthly_data_df, on = [:year, :firm_ID])
end

In [ ]:
#ch06_06
factor_data_df = @chain monthly_data_df begin
    @rtransform(:weight_return = :w_M * :R)
    @rsubset(:month_ID >= 13)
    groupby(:month_ID)
    @combine begin
        :R_F = mean(:R_F)
        :R_M = sum(skipmissing(:weight_return))
    end
    @rtransform(:R_Me = :R_M - :R_F)
end

In [ ]:
@chain factor_data_df begin
    plot(x = :R_Me, Geom.histogram)
end

In [ ]:
#ch06_09
@chain factor_data_df begin
    @rtransform(:gross_R_M = 1 + :R_M)
    @transform(:cumulative_gross_R_M = cumprod(:gross_R_M))
    plot(x = :month_ID, y = :cumulative_gross_R_M, Geom.line)
end

In [ ]:
#ch06_10
temp_df = @chain factor_data_df begin
    @rtransform(:gross_R_M = 1 + :R_M)
    @transform(:cumulative_gross_R_M = cumprod(:gross_R_M))
    @select(:month_ID, :cumulative_gross_R_M)
    push!([12, 1])
    @orderby(:month_ID)
    plot(x = :month_ID, y = :cumulative_gross_R_M, Geom.line)
end

In [ ]:
#ch06_11
annual_data_df = @chain annual_data_df begin
    groupby(:year)
    @transform(:ME_rank10 = cut(:lagged_ME, 10, labels=1:10))
end